# Importación de librerías

In [1]:
import warnings
warnings.filterwarnings('ignore') 

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import re # Expresiones regulares
from sklearn import preprocessing # LabelEncoder

In [3]:
data = pd.read_table("../Dataset/labeledEligibilitySample10k.csv", header = None)

In [4]:
print(data.shape)

(10000, 1)


### Se transforma la variable elegible (la salida) en 0 y 1

In [5]:
clin_trial = pd.DataFrame(np.array(data).reshape(10000,1), columns=['Description'])
clin_trial[:].head(10)

,Description
0,"18518,__label__0,study interventions are Erlot..."
1,"71331,__label__0,study interventions are Antib..."
2,"232342,__label__0,study interventions are Cycl..."
3,"450677,__label__0,study interventions are Pred..."
4,"410008,__label__0,study interventions are Inte..."
5,"307723,__label__0,study interventions are Erlo..."
6,"378773,__label__0,study interventions are Phar..."
7,"479580,__label__0,study interventions are Myco..."
8,"458795,__label__0,study interventions are gamm..."
9,"366518,__label__0,study interventions are Succ..."


In [6]:
clin_trial['Num'], clin_trial['ExtraPart'] = clin_trial['Description'].str.split(',', 1).str
clin_trial=clin_trial.drop(['Num'], axis=1)

clin_trial['Label'], clin_trial['TrashPart'] = clin_trial['ExtraPart'].str.split(',', 1).str
clin_trial['Eligible'] = clin_trial['Label'].str.extract('(\d)', expand=True)
clin_trial=clin_trial.drop(['Label'], axis=1)
clin_trial=clin_trial.drop(['ExtraPart'], axis=1)

clin_trial['Interventions'], clin_trial['Diagnoses'] = clin_trial['TrashPart'].str.split('.', 1).str
clin_trial=clin_trial.drop(['TrashPart'], axis=1)

clin_trial=clin_trial.drop(['Description'], axis=1)

clin_trial.head(10)

,Eligible,Interventions,Diagnoses
0,0,study interventions are Erlotinib Hydrochloride,recurrent midline lethal granuloma of the par...
1,0,study interventions are Antibodies,fallopian tube cancer diagnosis and periphera...
2,0,study interventions are Cyclosporine,recurrent adult diffuse mixed cell lymphoma d...
3,0,study interventions are Prednisolone phosphate,childhood immunoblastic large cell lymphoma d...
4,0,study interventions are Interferon-alpha,stage multiple myeloma diagnosis and serum pr...
5,0,study interventions are Erlotinib Hydrochloride,patients must have carcinoma of unknown prima...
6,0,study interventions are Pharmacological Study,hepatoblastoma diagnosis and two weeks must h...
7,0,study interventions are Mycophenolic Acid,recurrent grade one follicular lymphoma diagn...
8,0,study interventions are gamma-secretase/Notch ...,recurrent non small cell lung cancer diagnosi...
9,0,study interventions are Succinylcholine,untreated childhood medulloblastoma diagnosis...


In [7]:
# Se quita study interventions are puesto que está en todas las muestras de la varialbe Interventions
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace("study interventions are\s", "")
clin_trial.sample(10)

,Eligible,Interventions,Diagnoses
1839,0,PD-0325901,neurofibromatosis type one and growing or sym...
4734,0,Cetuximab,metastatic colorectal cancer diagnosis and me...
7872,1,Angiogenesis Inhibitors,glioblastoma diagnosis and clinically signifi...
7073,1,Etoposide phosphate,recurrent mediastinal thymic large cell cell ...
8521,1,Paclitaxel,breast cancer diagnosis and pre existing trea...
8164,1,Cyclophosphamide,hertwo positive breast cancer diagnosis and c...
8285,1,Methylprednisolone Hemisuccinate,non hodgkin lymphoma diagnosis and known alle...
1291,0,Tacrolimus,recurrent adult diffuse small cleaved cell ly...
5340,1,PF00562271,prostatic neoplasm diagnosis and clinically s...
1304,0,Paclitaxel,unspecified adult solid tumor protocol specif...


In [8]:
#Sirve para buscar palabras con esos caracteres

# for name in list(clin_trial.columns):
#     for sample in clin_trial[name]:
#         if re.findall('[+=*^#áÁéÉíÍóÓúÚ\[\]\$,.;:!@¡¿?\{\}\&`~\^\(\)|%_\-<>\'\"]', sample):
#             print(sample, "\n") ## Funciona para detectar caracteres especiales

In [9]:
# Se transforma todo a minúsculas
clin_trial['Interventions'] = clin_trial['Interventions'].str.lower() ##Todo a minúscula
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.lower()     ##Todo a minúscula

#Se reemplazan vocales con tildes en ambas variables
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[áäâà]', 'a', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[éêèë]', 'e', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[íïìî]', 'i', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[óôòö]', 'o', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('[úûùü]', 'u', regex=True)
clin_trial['Interventions'] = clin_trial['Interventions'].str.replace('["]', '', regex=True)

clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[áäâà]', 'a', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[éêèë]', 'e', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[íïìî]', 'i', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[óôòö]', 'o', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[úûùü]', 'u', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('^\s', '', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('["]', '', regex=True)
clin_trial['Diagnoses'] = clin_trial['Diagnoses'].str.replace('[;]', '', regex=True)



#clin_trial.sample(10)

In [10]:
# Total de categorías diferentes en la variable Interventions
len(np.unique(clin_trial['Interventions'])) 

1426

In [11]:
# Sirve para ver cuantos datos hay por categoría de interventions

datosInt = clin_trial['Interventions'].value_counts()

In [12]:
# Total de categorías diferentes en la variable Diagnoses
len(np.unique(clin_trial['Diagnoses']))

9786

In [13]:
datosDiag = clin_trial['Diagnoses'].value_counts() 
#datosDiag

## Codificación de la variable Interventions

In [14]:
labels = np.unique(clin_trial['Interventions'])

lb_interventions = preprocessing.LabelEncoder()
lb_interventions.fit(labels)
clin_trial['Interventions'] = lb_interventions.transform(clin_trial['Interventions']) 

In [15]:
#clin_trial

# Remover Stop Words

## Preliminares

In [16]:
# Se carga la libreria
from nltk.corpus import stopwords

# Se debe descargar el conjunto de 'Stop Words' la primera vez
import nltk
nltk.download('stopwords')

#Se carga la librería de lematización
#PONER conda install -c conda-forge spacy EN ANACONDA PROMPT
# Y python -m spacy download en_core_web_sm

#Se carga la librería de lematización
import spacy
nlp = spacy.load("en_core_web_sm")

#Se carga la librería de stemming y se inicializa el stemmer
import nltk
from nltk import SnowballStemmer
stemmer=SnowballStemmer('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Crear las palabras Token

## Lematización

"Dada una forma flexionada (es decir, en plural, en femenino, conjugada, etc), hallar el lema correspondiente. El lema es la forma que por convenio se acepta como representante de todas las formas flexionadas de una misma palabra. Es decir, el lema de una palabra es la palabra que nos encontraríamos como entrada en un diccionario tradicional: singular para sustantivos, masculino singular para adjetivos, infinitivo para verbos." -Wikipedia

In [17]:
#https://medium.com/qu4nt/reducir-el-número-de-palabras-de-un-texto-lematización-y-radicalización-stemming-con-python-965bfd0c69fa
toks = []

for sample in clin_trial['Diagnoses']:
    words = nlp(sample)
    lemma = [tok.lemma_.lower() for tok in words]
    #print(lemma)
    toks.append(lemma)
    

## Cargar las Stop Words

In [18]:
# Carga las Stop Words en inglés
stop_words = stopwords.words('english')

## Estaría bien revisar este conjunto 'english'. Pero supongo que es el más útil para nuestro caso también.
# Imprime algunas Stop Words
#stop_words[:100]

## Remover las Stop Words

In [19]:
tokens = []
for tok in toks:
    clean_word = [word for word in tok if word not in stop_words]
    tokens.append(clean_word)

## Stemming

"Es un método para reducir una palabra a su raíz o (en inglés) a un stem." - Wikipedia

In [20]:
stemms = []
for tok in tokens:
    var = [stemmer.stem(token) for token in tok]
    stemms.append(var)

# Bag of Words

In [21]:
def extract_words(sentence):
    words = []
    for i in sentence:
        if i not in words:
            words.append(i)
    return words

def getAll_words(data):
    words = []
    for i in data:
        wordsPerSentence = extract_words(i)
        for j in wordsPerSentence:
            if j not in words:
                words.append(j)
    return words

def wordFrequency(sentence, words):
    sentence_words = sentence
    # frequency word count
    bag = np.zeros(len(words))
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag)

def bagOfWords(data, words):
    bow = []
    for i in data:
        freq = wordFrequency(i, words)
        bow.append(freq)
    return bow

In [22]:
words = getAll_words(stemms)
bow = bagOfWords(stemms, words)

In [23]:
bowArray = np.asarray(bow)
wordsArray = np.asarray(words)
df = pd.DataFrame(data=bowArray, columns= wordsArray)

In [24]:
del clin_trial['Diagnoses']
clin_trial_full = pd.concat([clin_trial, df], axis=1)
clin_trial_full

,Eligible,Interventions,recurr,midlin,lethal,granuloma,paranas,sinus,nasal,caviti,...,kaposiform_hemangioendothelioma,khe,damag,peroxid,color,bleach,streak,perm,relax,oxid
0,0,464,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,349,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,1109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,1,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,1,1201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,1,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
clin_trial_full.to_hdf('bagOfWords&interventios.h5', key = 'clin_trial')

In [26]:
#stemms

In [27]:
bagOfWords = pd.read_hdf('bagOfWords&interventios.h5', 'clin_trial')

In [28]:
bagOfWords

,Eligible,Interventions,recurr,midlin,lethal,granuloma,paranas,sinus,nasal,caviti,...,kaposiform_hemangioendothelioma,khe,damag,peroxid,color,bleach,streak,perm,relax,oxid
0,0,464,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,349,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,1109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,1,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,1,1201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,1,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Ordenar y eliminar palabras

In [29]:
import operator

bowDict = {}

for column in bagOfWords:
    if column == 'Eligible' or column == 'Interventions':
        continue
    bowDict[column] = bagOfWords[column].sum() #Cuenta palabras

bowDict_sort = sorted(bowDict.items(), key=operator.itemgetter(1), reverse=True)

len(bowDict_sort)
# print(bowDict_sort)

5288

In [30]:
bowDict_sort2 = []
i = 0
for word in bowDict_sort:    
    if word[1] <= 20.0: #Para eliminar con un umbral
        continue
    bowDict_sort2.append(bowDict_sort[i][0])
    i = i + 1
del bowDict_sort2[0]

In [31]:
print(len(bowDict_sort2), len(bowDict_sort))

873 5288


# TF-IDF 
Term frequency – Inverse document frequency

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
#bowDict_sort2

In [34]:
stemmsT = [ ' '.join(stemms[item]) for item in range (len(stemms))] #Se transforma de lista de listas a una lista

In [35]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(stemmsT)

In [36]:
cols = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()

In [37]:
df = pd.DataFrame(data=denselist, columns= cols)

In [38]:
#df

In [39]:
aux = df
for word in df:
    if word not in bowDict_sort2:
        del aux[word]

In [40]:
aux.shape

(10000, 867)

In [41]:
df = aux

# Concatenación de dfs

In [42]:
clin_trial_full = pd.concat([clin_trial, df], axis=1)
clin_trial_full

,Eligible,Interventions,abdomin,abil,abl,abnorm,abscess,absenc,absolut,absorpt,...,within,without,woman,would,wound,write,year,york,zero,zone
0,0,464,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0,101,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.204349,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0,349,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0,1052,0.0,0.0,0.0,0.119754,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.099676,0.0
4,0,708,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,1109,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
9996,1,200,0.0,0.0,0.0,0.256094,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
9997,1,1201,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.210823,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
9998,1,939,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.366276,0.0,0.0,0.0,0.000000,0.0


# Guardado del modelo

In [43]:
clin_trial_full.to_hdf('bagOfWords10k_TFIDF-BoW.h5', key = 'clin_trial')

In [44]:
df_loaded = pd.read_hdf('bagOfWords10k_TFIDF-BoW.h5', 'clin_trial')

In [45]:
df_loaded

,Eligible,Interventions,abdomin,abil,abl,abnorm,abscess,absenc,absolut,absorpt,...,within,without,woman,would,wound,write,year,york,zero,zone
0,0,464,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0,101,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.204349,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0,349,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0,1052,0.0,0.0,0.0,0.119754,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.099676,0.0
4,0,708,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,1109,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
9996,1,200,0.0,0.0,0.0,0.256094,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
9997,1,1201,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.210823,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
9998,1,939,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.366276,0.0,0.0,0.0,0.000000,0.0


In [55]:
eligible = df_loaded['Eligible'].values #Para ver las palabras por clase

dict_0 = {}
dict_1 = {}

for col in df_loaded:
    if(col == 'Eligible' or col == 'Interventions'):
        continue
    ar = df_loaded[col].values
    condlist = [(eligible == "1") & (ar != 0.0), (eligible == "0") & (ar != 0.0)]
    choicelist = [1, 2]
    res = np.select(condlist, choicelist)
    
    dict_0[col] = len(res[res==2])
    dict_1[col] = len(res[res==1])
    
#     print(col + " tiene ", len(res[res==2]), " apariciones en la clase 0")
#     print(col + " tiene ", len(res[res==1]), " apariciones en la clase 1 \n")

# Algoritmo de selección

In [79]:
from sklearn.preprocessing import MinMaxScaler # Escala los datos
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import scikitplot as skplt 
from string import ascii_uppercase 
# import seaborn as sns
import qgrid
import time
from sklearn.externals import joblib # Para guardar el modelo
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [80]:
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression

In [81]:
clin_trial_values = clin_trial_full.values
Y = clin_trial_values[:, 0]
Y = Y.astype(int)
X = clin_trial_values[:, 1:]

In [87]:
def select_features(modelo, n_features, fwd, fltg):

    sfs = SFS(modelo, 
           k_features=n_features,
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0,
           n_jobs=-1)
    
    return sfs

In [83]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [84]:
def plot_roc(Xtest, Ytest, probs, xlabel):
    ns_probs = [0 for _ in range(len(Ytest))]
    
    probs = probs[:, 1]
    ns_auc = roc_auc_score(Ytest, ns_probs)
    auc = roc_auc_score(Ytest, probs)  

    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print('Logistic: ROC AUC=%.3f' % (auc))

    ns_fpr, ns_tpr, _ = roc_curve(Ytest, ns_probs)
    fpr, tpr, _ = roc_curve(Ytest, probs)   

    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(fpr, tpr, marker='.', label= xlabel)

    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [88]:
def seletion(features):
    
    fwd = True
    fltg = False
        
    #Para calcular el costo computacional
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,2])
    recall_list = np.zeros([4,2])
    f_list = np.zeros([4,2])
  
    #Implemetamos la metodología de validación 

    Errores = np.ones(4)
    j = 0
    model = LogisticRegression(C=1, solver='liblinear', max_iter = 200)
    #model = RandomForestClassifier(n_estimators=20, max_features = 2)#Son dos porque sino no se puede ejecutar con 3 variables
    
    for j in range(4):

        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25) # Modificar metodología de validación
        scaler = MinMaxScaler()#Escala entre 0 y 1
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        

        sf = select_features(model, features, fwd, fltg)

        #Complete el códpip install mlxtend  igo para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf =  sf.fit(Xtrain, Ytrain)

        X_train_sfs = sf.transform(Xtrain)
        X_test_sfs = sf.transform(Xtest)
        XN = sf.transform(X)

        # Fit the estimator using the new feature subset
        # and make a prediction on the test data
        model.fit(X_train_sfs, Ytrain)
        pred = model.predict(X_test_sfs)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        Errores[j] = classification_error(pred, Ytest)
        
    print(sf.k_feature_idx_)    
    return XN, X_train_sfs, X_test_sfs, str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(Errores)), str(np.std(Errores)), str(time.time()-tiempo_i)

In [89]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({'# de características seleccionadas' : pd.Series([800,700,600,500,400,300,200,100])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['# de características seleccionadas'], inplace=True)

for k in df_types.index:
    XN, XtrainN, XtestN, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = selection(k)
    print(k)
    df_types["Eficiencia"][k] = Acc
    df_types["Int_Eficiencia"][k] = IntAcc
    df_types["Sensibilidad"][k] = Sen
    df_types["Int_Sensibilidad"][k] = IntSen
    df_types["Precision"][k] = Pre
    df_types["Int_Precision"][k] = IntPre
    df_types["F-Score"][k] = f
    df_types["Int_F-Score"][k] = IntF
    df_types["Error_Prueba"][k] = error
    df_types["Int_error"][k] = stdError
    df_types["Tiempo de ejecución"][k] = tiempo
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 868 out of 868 | elapsed:   36.9s finished
Features: 867/800[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 867 out of 867 | elapsed:   34.1s finished
Features: 866/800[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 176 tas

Features: 848/800[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 848 out of 848 | elapsed:   36.7s finished
Features: 847/800[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 847 out of 847 | elapsed:   37.3s finished
Features: 846/800[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-

KeyboardInterrupt: 

In [ ]:
qgrid_widget.get_changed_df()